In [12]:
import pandas as pd
from pathlib import Path
import shutil

In [15]:
dirs = list(Path(".").glob("GSE*"))
dirs

[PosixPath('GSEhumanDHS')]

In [16]:
#metameta = []
for d in dirs: 
    print(d)
    meta = pd.read_csv(d / "meta.tsv", sep="\t")
    final = []
    for i in range(meta.shape[0]): 
        species_genome = meta.loc[i, "species"] + "_" + meta.loc[i, "genome"]
        species_dir = Path(species_genome)
        species_dir.mkdir(parents=True, exist_ok=True)
        tissue = meta.loc[i, "tissue"]
        assay = meta.loc[i, "assay"]
        final_filename = species_dir / f"{str(d)}_{tissue}_{assay}.bed.gz"
        shutil.copy(d / meta.loc[i, "filename"], final_filename)
        final.append(str(final_filename))
    meta["final"] = final
    metameta.append(meta)

GSEhumanDHS


In [27]:
len(metameta)

9

In [19]:
mm.to_csv("gse_meta.tsv", sep="\t", index=False)

In [20]:
mm

,tissue,assay,species,genome,filename,original,final
0,forelimb_bud_20,ATAC,gallus_gallus,galGal5,20_ATAC_gallus_gallus.bed.gz,GSM5017897_h20_ATAC_rep1_dpProfiles.narrow_pea...,gallus_gallus_galGal5/GSE164738_forelimb_bud_2...
1,forelimb_bud_105,ATAC,mus_musculus,GRCm38,105_ATAC_mus_musculus.bed.gz,GSM5017894_e105_ATAC_rep2_dpProfiles.narrow_pe...,mus_musculus_GRCm38/GSE164738_forelimb_bud_105...
2,forelimb_bud_975,ATAC,mus_musculus,GRCm38,975_ATAC_mus_musculus.bed.gz,GSM5017891_e975_ATAC_rep1_dpProfiles.narrow_pe...,mus_musculus_GRCm38/GSE164738_forelimb_bud_975...
3,forelimb_bud_22,ATAC,gallus_gallus,galGal5,22_ATAC_gallus_gallus.bed.gz,GSM5017899_h22_ATAC_rep1_dpProfiles.narrow_pea...,gallus_gallus_galGal5/GSE164738_forelimb_bud_2...
4,forelimb_bud_115,ATAC,mus_musculus,GRCm38,115_ATAC_mus_musculus.bed.gz,GSM5017896_e115_ATAC_rep2_dpProfiles.narrow_pe...,mus_musculus_GRCm38/GSE164738_forelimb_bud_115...
...,...,...,...,...,...,...,...
146,Umbilical Cord_Primary_2:Umbilicalcord,ATAC,homo_sapiens,GRCh38,Umbilical Cord_Primary_2_ATAC.bed.gz,ENCFF133TDW,homo_sapiens_GRCh38/GSEhumanDHS_Umbilical Cord...
147,"Umbilical_Primary_0:vein,umbilical,endothelial",ATAC,homo_sapiens,GRCh38,Umbilical_Primary_0_ATAC.bed.gz,ENCFF011FHJ,homo_sapiens_GRCh38/GSEhumanDHS_Umbilical_Prim...
148,"Uterus_Cancer_1:adherent,cervical,adenocarcinoma",ATAC,homo_sapiens,GRCh38,Uterus_Cancer_1_ATAC.bed.gz,"ENCFF503PAE,ENCFF782HVG,ENCFF309DYE",homo_sapiens_GRCh38/GSEhumanDHS_Uterus_Cancer_...
149,"Uterus_Cancer_2:adherent,cervical,adenocarcinoma",ATAC,homo_sapiens,GRCh38,Uterus_Cancer_2_ATAC.bed.gz,ENCFF898KEE,homo_sapiens_GRCh38/GSEhumanDHS_Uterus_Cancer_...


## Ianimal

In [25]:
dirs = [p for p in Path("ianimal").glob("*") if p.is_dir()]
dirs

[PosixPath('ianimal/Bos_taurus'),
 PosixPath('ianimal/Sus_scrofa'),
 PosixPath('ianimal/Macaca_mulatta'),
 PosixPath('ianimal/Mus_musculus'),
 PosixPath('ianimal/Capra_hircus'),
 PosixPath('ianimal/Ovis_aries'),
 PosixPath('ianimal/Oryctolagus_cuniculus')]

In [26]:
species_genome_dict = { 
    "Sus_scrofa" : "Sscrofa11.1", 
    "Bos_taurus" : "ARS-UCD1.2",
    "Ovis_aries" : "Oar_rambouillet_v1.0",
    "Capra_hircus" : "ARS1",
    "Mus_musculus" : "GRCm39",
    "Macaca_mulatta" : "Mmul_10",
    "Oryctolagus_cuniculus" : "OryCun2.0"
}


In [52]:
import re
def get_meta(m): 
    return [ m.group(i) for i in [1,2] ]

for d in dirs: 
    files = list(d.glob("*.bed.gz") )

    matches = []
    for f in files: 
        matches.append( 
            get_meta(re.search(r"(.*)_([-A-Za-z0-9]+)\.bed\.gz", str(f.name)))
        )
    meta = pd.DataFrame(matches, columns = ["tissue", "assay"])
    species = str(d.name).lower()
    genome = species_genome_dict[str(d.name)]
    meta["species"] = species
    meta["genome"] = genome
    meta["filename"] = [ f.name for f in files ]

    species_dir = Path(f"{species}_{genome}")
    species_dir.mkdir(parents=True, exist_ok=True)
    final = []
    
    for i in range(meta.shape[0]): 
        tissue = meta.loc[i, "tissue"]
        assay = meta.loc[i, "assay"]
        final_filename = species_dir / f"ianimal_{tissue}_{assay}.bed.gz"
        shutil.copy(d / meta.loc[i, "filename"], final_filename)
        #print(d / meta.loc[i, "filename"], final_filename)
        final.append(str(final_filename))
    meta["final"] = final
    meta["original"] = meta["filename"]
    
    metameta.append(meta)
    #break

meta

,tissue,assay,species,genome,filename,final,original
0,brain,H3K4me3,oryctolagus_cuniculus,OryCun2.0,brain_H3K4me3.bed.gz,oryctolagus_cuniculus_OryCun2.0/ianimal_brain_...,brain_H3K4me3.bed.gz
1,testis,H3K4me3,oryctolagus_cuniculus,OryCun2.0,testis_H3K4me3.bed.gz,oryctolagus_cuniculus_OryCun2.0/ianimal_testis...,testis_H3K4me3.bed.gz
2,brain,H3K4me1,oryctolagus_cuniculus,OryCun2.0,brain_H3K4me1.bed.gz,oryctolagus_cuniculus_OryCun2.0/ianimal_brain_...,brain_H3K4me1.bed.gz
3,testis,H3K27ac,oryctolagus_cuniculus,OryCun2.0,testis_H3K27ac.bed.gz,oryctolagus_cuniculus_OryCun2.0/ianimal_testis...,testis_H3K27ac.bed.gz
4,muscle,H3K4me1,oryctolagus_cuniculus,OryCun2.0,muscle_H3K4me1.bed.gz,oryctolagus_cuniculus_OryCun2.0/ianimal_muscle...,muscle_H3K4me1.bed.gz
5,liver,H3K4me3,oryctolagus_cuniculus,OryCun2.0,liver_H3K4me3.bed.gz,oryctolagus_cuniculus_OryCun2.0/ianimal_liver_...,liver_H3K4me3.bed.gz
6,brain,H3K27ac,oryctolagus_cuniculus,OryCun2.0,brain_H3K27ac.bed.gz,oryctolagus_cuniculus_OryCun2.0/ianimal_brain_...,brain_H3K27ac.bed.gz
7,muscle,H3K27ac,oryctolagus_cuniculus,OryCun2.0,muscle_H3K27ac.bed.gz,oryctolagus_cuniculus_OryCun2.0/ianimal_muscle...,muscle_H3K27ac.bed.gz
8,liver,H3K4me1,oryctolagus_cuniculus,OryCun2.0,liver_H3K4me1.bed.gz,oryctolagus_cuniculus_OryCun2.0/ianimal_liver_...,liver_H3K4me1.bed.gz
9,interscapular fat,ATAC,oryctolagus_cuniculus,OryCun2.0,interscapular fat_ATAC.bed.gz,oryctolagus_cuniculus_OryCun2.0/ianimal_inters...,interscapular fat_ATAC.bed.gz


In [53]:
mm = pd.concat(metameta, axis = 0)

In [55]:
mm.to_csv("all_meta.tsv", sep="\t", index=False)

In [57]:
genome_urls = pd.read_csv("final_data/genome_urls.txt", sep=" ")
genome_urls

,genome,url
0,Oar_rambouillet_v1.0,https://ftp.ensembl.org/pub/release-110/fasta/...
1,ARS1,https://ftp.ensembl.org/pub/release-112/fasta/...
2,GRCm38,ftp://ftp.ensembl.org/pub/release-102/fasta/mu...
3,GRCm39,https://ftp.ensembl.org/pub/release-112/fasta/...
4,Mmul_10,https://ftp.ensembl.org/pub/release-112/fasta/...
5,OryCun2.0,https://ftp.ensembl.org/pub/release-112/fasta/...
6,Sscrofa11.1,https://ftp.ensembl.org/pub/release-112/fasta/...
7,Sscrofa11.2,https://ftp.ensembl.org/pub/release-112/fasta/...
8,Xenopus_tropicalis_v9.1,https://ftp.ensembl.org/pub/release-106/fasta/...
9,galGal5,https://hgdownload.soe.ucsc.edu/goldenPath/gal...


In [58]:
result = pd.merge(mm, genome_urls, on='genome', how='left') mm

,tissue,assay,species,genome,filename,original,final
0,forelimb_bud_20,ATAC,gallus_gallus,galGal5,20_ATAC_gallus_gallus.bed.gz,GSM5017897_h20_ATAC_rep1_dpProfiles.narrow_pea...,gallus_gallus_galGal5/GSE164738_forelimb_bud_2...
1,forelimb_bud_105,ATAC,mus_musculus,GRCm38,105_ATAC_mus_musculus.bed.gz,GSM5017894_e105_ATAC_rep2_dpProfiles.narrow_pe...,mus_musculus_GRCm38/GSE164738_forelimb_bud_105...
2,forelimb_bud_975,ATAC,mus_musculus,GRCm38,975_ATAC_mus_musculus.bed.gz,GSM5017891_e975_ATAC_rep1_dpProfiles.narrow_pe...,mus_musculus_GRCm38/GSE164738_forelimb_bud_975...
3,forelimb_bud_22,ATAC,gallus_gallus,galGal5,22_ATAC_gallus_gallus.bed.gz,GSM5017899_h22_ATAC_rep1_dpProfiles.narrow_pea...,gallus_gallus_galGal5/GSE164738_forelimb_bud_2...
4,forelimb_bud_115,ATAC,mus_musculus,GRCm38,115_ATAC_mus_musculus.bed.gz,GSM5017896_e115_ATAC_rep2_dpProfiles.narrow_pe...,mus_musculus_GRCm38/GSE164738_forelimb_bud_115...
...,...,...,...,...,...,...,...
8,liver,H3K4me1,oryctolagus_cuniculus,OryCun2.0,liver_H3K4me1.bed.gz,liver_H3K4me1.bed.gz,oryctolagus_cuniculus_OryCun2.0/ianimal_liver_...
9,interscapular fat,ATAC,oryctolagus_cuniculus,OryCun2.0,interscapular fat_ATAC.bed.gz,interscapular fat_ATAC.bed.gz,oryctolagus_cuniculus_OryCun2.0/ianimal_inters...
10,testis,H3K4me1,oryctolagus_cuniculus,OryCun2.0,testis_H3K4me1.bed.gz,testis_H3K4me1.bed.gz,oryctolagus_cuniculus_OryCun2.0/ianimal_testis...
11,liver,H3K27ac,oryctolagus_cuniculus,OryCun2.0,liver_H3K27ac.bed.gz,liver_H3K27ac.bed.gz,oryctolagus_cuniculus_OryCun2.0/ianimal_liver_...


In [ ]:
al